# 参数初始化

**本章内容作为补充内容，可以后期有需要使用别人已经预训练好了的模型再回来翻**


对于PyTorch中的各种网络层，我们会需要对参数进行初始化的操作，这里我们以 nn.Linear() 为例子：

```
class Linear(Module):
    def __init__(self, in_features, out_features, bias=True):
    ...
        self.reset_parameters()
    ...
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
```

上述代码摘自 PyTorch [torch.nn.Linear()的源码](http://pytorch.org/docs/master/_modules/torch/nn/modules/linear.html#Linear)

从源代码中可以看到，PyTorch在我们实例化某层的时候，就帮我们自动完成了初始化的操作。但是，有时候，我们可能需要一些不一样的初始化。

比如使用 xavier 方法 来进行初始化，我们该怎么做呢？

在PyTorch中有一个 torch.nn.init 这个包就是用来帮我们完成初始化的。

这个包提供了很多初始化的方法：

* torch.nn.init.constant(tensor, val)
* torch.nn.init.normal(tensor, mean=0, std=1)
* torch.nn.init.xavier_uniform(tensor, gain=1)

详情 http://pytorch.org/docs/master/nn.html#torch-nn-init

In [1]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self, in_dim, n_class):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_class)
        
        # 参数初始化函数
        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                nn.init.xavier_normal(p.weight.data)
            elif isinstance(p, nn.Linear):
                nn.init.normal(p.weight.data)

    # 向前传播
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


# 使用预训练的AlexNet模型来对CIFAR10进行训练

现在，假设，我们知道AlexNet是个好用的模型，我们想用它来对CIFAR10进行训练，但是我们不想自己训练，想用别人训练好的，我们该怎么做呢？

分三步：

1、加载预训练模型

2、对模型结构进行更改

3、重新训练，我们需要进行训练的那几层

In [ ]:
from torchvision import models, transforms, datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.optim as optim

In [ ]:
# -----------------get Alexnet model-------------------------
def getAlexNet(DOWNLOAD=True):
    alexnet = models.alexnet(pretrained=DOWNLOAD)
    return alexnet

# -----------------revise the AlexNet class--------------------------
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
        )
        self.fc = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        x = self.fc(x)
        return x


def to_var(x):
    x = Variable(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x


DOWNLOAD = True
# 下载预训练好的AlexNet模型
pre_alexnet = getAlexNet(DOWNLOAD)
pretrain_dict = pre_alexnet.state_dict()

# 因为 CIFAR10 只有10个种类，我们设置 num_classes=10
alexnet = AlexNet(10)
alexnet_dict = alexnet.state_dict()
pretrained_dict = {k: v for k, v in pretrain_dict.items() if k in alexnet_dict}

# 更新我们自己设置的Alexnet网络的权重
alexnet_dict.update(pretrained_dict)
# 将这些权重加载到模型中。
alexnet.load_state_dict(alexnet_dict)

# 使用GPU
if torch.cuda.is_available():
    alexnet = alexnet.cuda()

# get Dataset
transform = transforms.Compose([
    transforms.Scale(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = datasets.CIFAR10('./data', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10('./data', train=False, transform=transform, download=False)

train_data_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=256, shuffle=True)

# optim
learning_rate = 0.0001
num_epoches = 5
criterion = nn.CrossEntropyLoss()
# 训练的时候，我们只更新 fc 层的参数
optimizer = optim.Adam(alexnet.fc.parameters(), lr=learning_rate)

# training
alexnet.train()
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    runnin_acc = 0.0
    running_loss = 0.0
    for data, label in train_data_loader:
        img = to_var(data)
        label = to_var(label)
        out = alexnet(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        accuracy = (pred == label).float().mean()
        runnin_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Acc: {:.6f}".format(runnin_acc / len(train_data)))

# evaluation
alexnet.eval()
runnin_acc = 0.0
for data, label in test_data_loader:
    img = to_var(data)
    label = to_var(label)
    out = alexnet(img)
    loss = criterion(out, label)
    _, pred = torch.max(out, 1)
    num_correct = (pred == label).sum()
    accuracy = (pred == label).float().mean()
    runnin_acc += num_correct.data[0]
print("Acc: {:.6f}".format(runnin_acc / len(test_data)))


print("Done!")
